In [2]:
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import usefulFunctions.exergyFunctions as ef
import usefulFunctions.thermoFunctions as tf

In [3]:
gasconst = PropsSI('gas_constant','T',298.15,'P',101325,'PR::H2') # getting the gas constant from coolprop

8.3144598


In [4]:
chemExergyPath = 'D:/googledrive/DLR works/Chem exergies.csv' # database of chemical exergies

In [5]:
ech_df = pd.read_csv(chemExergyPath, delimiter=';', index_col=0)

In [6]:
ech_df

,State,Standard Chem Exergy (kJ/mol)
Al,s,795.7
Al4C3,s,4216.2
AlCl3,s,352.2
Al2O3,s. α corundum,15
Al2O3•H2O,s. boermite,9.4
...,...,...
ZnO,s,22.9
Zn(OH)2,s. β,25.7
ZnS,s. sphalerite,747.6
ZnSO4,s,82.3


In [7]:
def ech_stream(x_input, c_input):
    """
    Function for calculating molar chemical exergy of a gas mixture 
    based on the chosen chemical exergy database
    
    x_input: list of molar fractions of the species in the stream
    c_input: list of species names (example: O2, CO2) as a string
    
    returns chemical exergy in J/s
    """
    Tenv = 298.15 # K
    Penv = 100000 # Pa
    x_0 = np.zeros(len(c_input))
    Ex_ch_c = np.zeros(len(c_input))
    x_0_H2O = (0.03169/1) # from Raoult's law
    
    # flash calculation for water
    if 'H2O' in c_input:
        H2O = c_input.index('H2O')
        if x_input[H2O] <= x_0_H2O:
            rvapor = 1
            x_H2Obase = x_input[H2O]
        else:
            x_H2Obase = x_0_H2O
            rvapor = ((1-x_input[H2O])/(1-x_H2Obase)) # ratio of the part of the stream that is in gas/vapour at dead state
        for c in range(len(c_input)):
            if c_input[c] != 'H2O':
                x_0[c] = x_input[c]*(1-x_H2Obase)/(1-(x_input[H2O]))
            elif x_input[c]==0:
                x_0[c] = 0
            else:
                x_0[c] = x_H2Obase # saturated water vapour
                
        rliquid = (1-rvapor) # ratio of stream that condenses at dead state
        r_H2O = rliquid + rvapor*x_0[H2O]
        
        # chemical exergy calculation
        for c in range(len(c_input)):
            species = c_input[c]
            if x_input[c] == 0:
                Ex_ch_c[c] = 0
            else:
                if species!='H2O':
                    Ex_ch_c[c] = float(ech_df.loc[species]['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c])
                else:
                    Ex_ch_c[c] = (x_0[c]*rvapor*(float(ech_df.loc['H2O']['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c]))) + rliquid*float(ech_df.loc['H2O (L)']['Standard Chem Exergy (kJ/mol)'])*1000/r_H2O
    else:
        x_0 = x_input
        for c in range(len(c_input)):
            species = c_input[c]
            if x_input[c] == 0:
                Ex_ch_c[c] = 0
            else:
                Ex_ch_c[c] = float(ech_df.loc[species]['Standard Chem Exergy (kJ/mol)'])*1000 + gasconst*Tenv*np.log(x_0[c])
    
    Ex_ch = np.sum(x_input*Ex_ch_c)
    
    return Ex_ch
        
    # use float(ech_df.loc['O2']['Standard Chem Exergy (kJ/mol)']) to obtain value

In [8]:
# calculates exergy of stream using coolprop

def e_CP(name): # input stream name as string with ''
    """
    Function for calculating total exergy of a stream using CoolProp
    
    name: name of stream
    
    returns total exergy of a stream in J/s
    """
    species_list = []
    xlist = []
    for k in range(nSpecies):
        C = d[name]['C'+str(k+1)]
        c_short = spec[C]
        species_list.append((c_short))
        x_stream = float(d[name]['Molar Fraction (Vapor) - '+C+' /-'])
        xlist.append((x_stream))
    m_stream = float(d[name]['Molar Flow /kmol/h']) * (1000/3600) # mass flow (mol/s)
    ephlist = []
    hlist = []
    slist = []
    for k in range(nSpecies):
        h_c = xlist[k]*PropsSI('Hmolar','T|gas',float(d[name]['Temperature /C'])+Tenv,'P',float(d[name]['Pressure /bar'])*1e5,'PR::'+spec[d[name]['C'+str(k+1)]])
        henv_c = xlist[k]*PropsSI('Hmolar','T|gas',Tenv,'P',Penv,'PR::'+spec[d[name]['C'+str(k+1)]])
        s_c = xlist[k]*PropsSI('Smolar','T|gas',float(d[name]['Temperature /C'])+Tenv,'P',float(d[name]['Pressure /bar'])*1e5,'PR::'+spec[d[name]['C'+str(k+1)]])
        senv_c = xlist[k]*PropsSI('Smolar','T|gas',Tenv,'P',Penv,'PR::'+spec[d[name]['C'+str(k+1)]])
        h = h_c - henv_c # enthalpy difference between system and environment
        hlist.append((h))
        s = s_c - senv_c
        slist.append((s))
    EPHCP = sum(hlist) - Tenv*sum(slist) # calculating physical exergy
    ECH = ech_stream(xlist,species_list) # calculating chemical exergy using ech_stream function
    Etot = (ECH + EPHCP) * m_stream
    return Etot



In [11]:
filePath = "D:/googledrive/DLR works/SOC fuel cell results but xml.xml"

In [12]:
tree = ET.parse(source =filePath) # Get ElementTree
root = tree.getroot() # Extract Data
root # Show

<Element 'Objects' at 0x000001D75FDF1450>

In [13]:
objects = root.findall("Object") #Extract Objects into a list
objects

[<Element 'Object' at 0x000001D75FDF14A0>,
 <Element 'Object' at 0x000001D75FDFA860>,
 <Element 'Object' at 0x000001D75FDFFC20>,
 <Element 'Object' at 0x000001D75FE0B040>,
 <Element 'Object' at 0x000001D75FE11400>,
 <Element 'Object' at 0x000001D75FE114A0>]

In [14]:
test2 = objects[0].findall("Property") # Extract properties of first object into a list
test2

[<Element 'Property' at 0x000001D75FDF14F0>,
 <Element 'Property' at 0x000001D75FDF1540>,
 <Element 'Property' at 0x000001D75FDF1590>,
 <Element 'Property' at 0x000001D75FDF15E0>,
 <Element 'Property' at 0x000001D75FDF1630>,
 <Element 'Property' at 0x000001D75FDF16D0>,
 <Element 'Property' at 0x000001D75FDF1720>,
 <Element 'Property' at 0x000001D75FDF1770>,
 <Element 'Property' at 0x000001D75FDF17C0>,
 <Element 'Property' at 0x000001D75FDF1810>,
 <Element 'Property' at 0x000001D75FDF18B0>,
 <Element 'Property' at 0x000001D75FDF1950>,
 <Element 'Property' at 0x000001D75FDF19F0>,
 <Element 'Property' at 0x000001D75FDF1A90>,
 <Element 'Property' at 0x000001D75FDF1B30>,
 <Element 'Property' at 0x000001D75FDF1BD0>,
 <Element 'Property' at 0x000001D75FDF1C70>,
 <Element 'Property' at 0x000001D75FDF1D10>,
 <Element 'Property' at 0x000001D75FDF1D60>,
 <Element 'Property' at 0x000001D75FDF1DB0>,
 <Element 'Property' at 0x000001D75FDF1E00>,
 <Element 'Property' at 0x000001D75FDF1E50>,
 <Element 

In [15]:
# dictionary for short and long names of species
spec = {'Nitrogen':'N2', 'Hydrogen':'H2', 'Oxygen':'O2', 'Water':'H2O', 'Carbon Dioxide':'CO2', 'Carbon Monoxide':'CO', 'Ammonia':'NH3', 'Methane':'CH4'}

In [22]:
# xml extraction + stream exergy calculation 

tree = ET.parse(source =filePath) # Get ElementTree
root = tree.getroot() # Extract Data
objects = root.findall("Object") #Extract Objects into a list

Tenv = 298.15 # K
Penv = 100000 # Pa, can use 101325?
d = {} # put results into this dictionary
st = {} # dict for stream names
nSpecies = 4 # number of species (CHANGE THIS MANUALLY ACCORDING TO THE SYSTEM)
a=0

for i in range(len(objects)):

    print(objects[i].attrib) # Print attributes of the object: name and type
    props = objects[i].findall("Property") # Get list of properties for each object
    d[objects[i].attrib['name']] = {"type": objects[i].attrib['type']}
    p = len(props)
    j=0
        
    if 'Material Stream' in objects[i].attrib['type']:
        st_key = 'st_'+str(a+1)
        st_val = objects[i].attrib['name']
        st[st_key] = st_val
        a+=1
    
    while j <p:
        if props[j].attrib['units']=='': #Change units
            props[j].attrib['units']='-'

        if 'Molar Fraction' in props[j].attrib['name']:
            print('----------')
            for k in range(nSpecies):
                print(str(j+k+1) + ' -> ' + props[j].attrib['name'] + ' - ' + props[j+k+1].attrib['name'] + ' /' + props[j].attrib['units'] + ': ' + props[j+k+1].attrib['value'])
                key = props[j].attrib['name'] + ' - ' + props[j+k+1].attrib['name'] + ' /' + props[j].attrib['units']
                val = props[j+k+1].attrib['value']
                d[objects[i].attrib['name']][key] = val
            print('----------')
            j+=nSpecies+1
        else:
            print(str(j) + ' -> ' + props[j].attrib['name'] + ' /' + props[j].attrib['units'] + ': ' + props[j].attrib['value'])
            key = props[j].attrib['name'] + ' /' + props[j].attrib['units']
            val = props[j].attrib['value']
            d[objects[i].attrib['name']][key] = val
            j+=1
    
    if 'Material Stream' in objects[i].attrib['type']:
        # extracting names of components in the material streams
        for k in range(nSpecies):
            print(str(p+k) + ' -> Species ' + str(k+1) + ': ' + props[6+k].attrib['name'])
            key = 'C'+str(k+1)
            val = props[6+k].attrib['name']
            d[objects[i].attrib['name']][key] = val
        print('----------')    
        species_list = []
        xlist = []
        for k in range(nSpecies):
            C = d[objects[i].attrib['name']]['C'+str(k+1)]
            c_short = spec[C]
            species_list.append((c_short))
            x_stream = float(d[objects[i].attrib['name']]['Molar Fraction (Vapor) - '+C+' /-'])
            xlist.append((x_stream))
        # calculation of exergy of each streams using the available simulation data
        m_stream = float(d[objects[i].attrib['name']]['Molar Flow /mol/s']) # mass flow (mol/s)
        h_stream = float(d[objects[i].attrib['name']]['Molar Enthalpy (Mixture) /kJ/kmol'])
        s_stream = float(d[objects[i].attrib['name']]['Molar Entropy (Mixture) /kJ/[kmol.K]'])
        ECH = ech_stream(xlist,species_list) # J/mol
        EPH = h_stream - Tenv*s_stream # J/mol
        E_stream = (ECH + EPH) * m_stream # J/s
        print(str(p+k+1) + ' -> Exergy of Stream /J/s: ' + str(E_stream))
        print('----------')
        key = 'Exergy of Stream /J/s'
        val = str(E_stream)
        d[objects[i].attrib['name']][key] = val    
        
    print()

{'name': 'water_in', 'type': 'Material Stream'}
0 -> Temperature /K: 1073.15
1 -> Pressure /Pa: 101325
2 -> Mass Flow /kg/s: 1
3 -> Molar Flow /mol/s: 60.9186
4 -> Volumetric Flow /m3/s: 5.36419
----------
6 -> Mixture Molar Fraction - Hydrogen /-: 0.1
7 -> Mixture Molar Fraction - Oxygen /-: 0
8 -> Mixture Molar Fraction - Water /-: 0.9
9 -> Mixture Molar Fraction - Nitrogen /-: 0
----------
10 -> Density (Mixture) /kg/m3: 0.186422
11 -> Molecular Weight (Mixture) /kg/kmol: 16.4153
12 -> Specific Enthalpy (Mixture) /kJ/kg: 1728.03
13 -> Specific Entropy (Mixture) /kJ/[kg.K]: 3.78548
14 -> Molar Enthalpy (Mixture) /kJ/kmol: 28366.1
15 -> Molar Entropy (Mixture) /kJ/[kmol.K]: 62.14
16 -> Thermal Conductivity (Mixture) /W/[m.K]: 0.143673
----------
18 -> Molar Fraction (Vapor) - Hydrogen /-: 0.1
19 -> Molar Fraction (Vapor) - Oxygen /-: 0
20 -> Molar Fraction (Vapor) - Water /-: 0.9
21 -> Molar Fraction (Vapor) - Nitrogen /-: 0
----------
----------
23 -> Molar Fraction (Overall Liquid) 

In [23]:
print(st)

{'st_1': 'water_in', 'st_2': 'air_in', 'st_3': 'water_out', 'st_4': 'air_out'}


In [25]:
d.keys()

dict_keys(['water_in', 'air_in', 'water_out', 'air_out', 'ESTR-01', 'ESTR-02'])

In [26]:
df = pd.DataFrame.from_dict(d)

In [27]:
df

,water_in,air_in,water_out,air_out,ESTR-01,ESTR-02
type,Material Stream,Material Stream,Material Stream,Material Stream,Energy Stream,Energy Stream
Temperature /K,1073.15,1073.15,1073.15,1073.15,NaN,NaN
Pressure /Pa,101325,101325,101325,101325,NaN,NaN
Mass Flow /kg/s,1,1,1.00829,0.991709,NaN,NaN
Molar Flow /mol/s,60.9186,34.6616,60.9191,34.4025,NaN,NaN
Volumetric Flow /m3/s,5.36419,3.05213,5.36423,3.02931,NaN,NaN
Mixture Molar Fraction - Hydrogen /-,0.1,0,0.0915013,0,NaN,NaN
Mixture Molar Fraction - Oxygen /-,0,0.21,0,0.204052,NaN,NaN
Mixture Molar Fraction - Water /-,0.9,0,0.908499,0,NaN,NaN
Mixture Molar Fraction - Nitrogen /-,0,0.79,0,0.795948,NaN,NaN


In [31]:
# exergy balance with function

def exergybalance(stream_in, stream_out):
    """
    Function for calculating exergy destruction and exergy balance of a component using exergy of streams
    stream_in:  list of streams going into the component 
    stream_out: list of streams going out of the component
    the format is (['stream_1','stream_2'],['stream_3','stream_4'])
    
    WARNING: check the units of each property of the streams which are used for calling the dictionaries
    """
    E_in = 0
    E_out = 0
    for i in range(len(objects)):
        if objects[i].attrib['name'] in stream_in:
            if objects[i].attrib['type'] == 'Energy Stream':
                E_in = E_in + float(df[objects[i].attrib['name']]['Energy Flow /kW'])*1000 # J/s
            else:
                E_in = E_in + float(df[objects[i].attrib['name']]['Exergy of Stream /J/s'])
        if objects[i].attrib['name'] in stream_out:
            if objects[i].attrib['type'] == 'Energy Stream':
                E_out = E_out + float(df[objects[i].attrib['name']]['Energy Flow /kW'])*1000 # J/s
            else:
                E_out = E_out + float(df[objects[i].attrib['name']]['Exergy of Stream /J/s'])
        else:
            pass

    E_D = E_in - E_out

    des = print('The exergy destruction of the system is',E_D,'J/s')

    if E_D > 0 and -0.1 < E_in - E_out - E_D < 0.1:
        truct = print('The exergy balance is fulfilled for this system.')
    else:
        truct = print('The exergy balance is not fulfilled.')
    
    destruct = print(des,truct)
        
    return E_D # ASK, maybe return the ED value + boolean true/false for balance/not?

In [32]:
exergybalance(['water_in','air_in','ESTR-01'],['water_out','air_out'])

The exergy destruction of the system is 110578.53132885508 J/s
The exergy balance is fulfilled for this system.
None None


110578.53132885508